<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://d8a-88.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 12pt;"><strong>Economic Models</strong>, Fall 2019<br>
            Dr. Eric Van Dusen</p></td></tr>
</table>

# Project 3: Econometrics and Data Science



This project focuses on the application of the data science techniques from lecture. You will practice single variable ordinary least squares regression in the Data 8 style, go through a guided introduction to multivariate OLS using the package `statsmodels`, and finally create your own multivariate OLS model.

After this project, you should be able to

1. Write and apply the necesssary functions to perform single variable OLS
2. Use the `statsmodels` package to create multivariate OLS models
3. Understand how to quantitatively evaluate models using the root-mean-squared error
4. Look for and use relationships between variables to select features for regression

In [ ]:
from datascience import *
import numpy as np
import statsmodels.api as sm
from ipywidgets import interact, Dropdown, IntSlider
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore')
import otter
grader = otter.Notebook()
plt.style.use('seaborn-muted')
%matplotlib inline
plt.rcParams["figure.figsize"] = [10,7]

def get_dummies(tbl, col, drop=True):
    """Creates dummy variables for a column of a table"""
    values = np.unique(tbl.column(col))
    for val in values:
        encoding = tbl.apply(lambda s: int(s == val), col)
        tbl = tbl.with_column(col + "_" + str(val), encoding)
    if drop:
        tbl = tbl.drop(col)
    return tbl

In this project, we will be working with data on credit card defaults and billing. The data covers April to September 2005, with one row for each cardholder. It has the following columns:

| Column | Description |
|-----|-----|
| `credit` | Total amount of credit |
| `sex` | Cardholder sex |
| `education` | Cardholder education level |
| `martial_status` | Cardholder marital status |
| `age` | Cardholder age |
| `bill_{month}05` | Bill amount for specific month |
| `paid_{month}05` | Amount paid in specified month |
| `default` | Whether the cardholder defaulted |

In the cell below, we load the dataset.

In [ ]:
defaults_raw = Table.read_table("defaults.csv")
defaults_raw

**Question 0.1:** Which of the columns in `defaults_raw` would we need dummies for in orde rto use in an OLS model? Assign `ans_0_1` to an array of these column _labels_.

In [ ]:
ans_0_1 = ...
ans_0_1

In [ ]:
grader.check("q0_1")

In order to use the columns you chose, we will need to create dummies for them. In lecture, we showed a function (defined in the imports cell) that will get dummies for a variable for you.

**Question 0.2:** Use the `get_dummies` function to get dummies for the variables you listed in Question 0.1. _Make sure you drop the original columns using the `drop` argument._

In [ ]:
defaults_raw = get_dummies(...)
defaults_raw = get_dummies(...)
defaults = get_dummies(...)
defaults

In [ ]:
grader.check("q0_2")

## Part 1: Single Variable OLS

We'll start by doing some single variable linear regression, ala Data 8. To begin, recall that we can model $y$ based on $x$ using the form

$$\Large
\hat{y} = \alpha + \beta x
$$

We can define the **correlation coefficient** of two values to be the mean of the product of their values in standard units.

**Question 1.1:** Complete the `corr` function below to compute the correlation coefficient of two arrays `x` and `y` based on the formula

$$\Large
r = \text{mean} \left ( x_\text{SU} \cdot y_\text{SU} \right )
$$

_Hint:_ You may find the `su` function, which converts an array to standard units, helpful.

In [ ]:
def su(arr):
    """Converts array arr to standard units"""
    return (arr - np.mean(arr)) / np.std(arr)

def corr(x, y):
    """Calculates the correlation coefficient of two arrays"""
    return ...

In [ ]:
grader.check("q1_1")

From this $r$ value that we have calculated above, we can compute the slope $m$ and intercept $b$ of the best-fit line using the formulas below.

$$\Large
\beta = r \frac{\text{SD} (y)}{\text{SD}(x)}
\qquad \text{ and } \qquad
\alpha = \text{mean} (y) - \beta \cdot \text{mean}(x)
$$

**Question 1.2:** Fill in the `slope` and `intercept` functions below which compute the values of $m$ and $b$ for the line of best fit that predicts `y` based on `x`.

_Hint:_ You may find your `slope` function useful in `intercept`.

In [ ]:
def slope(x, y):
    """Computes the slope of the best-fit line of y based on x"""
    return ...

def intercept(x, y):
    """Computes the intercept of the best-fit line of y based on x"""
    return ...

In [ ]:
grader.check("q1_2")

Now let's look at how we can predict the `bill_sep05` column based on some other column of our data. We'll start by looking at the `credit` as the explanatory variable. To use our functions above, we must extract the values of each column as arrays, which we define below as `x` and `y`. We then compute the fitted values `y_hat` using the slope-intercept formula and plot the results.

In [ ]:
x = defaults.column("credit")
y = defaults.column("bill_sep05")

alpha = slope(x, y)
beta = intercept(x, y)

y_hat = alpha * x + beta

plt.scatter(x, y, color="tab:blue", alpha=0.3)
plt.plot(x, y_hat, color="tab:red")
plt.title("Predict September bill with credit");

Obviously, we can see that our best-fit line does not predict perfectly. There are plenty of points in the scatterplot that do not fall on the line. But how do we quantify the error of our model? There are many so-called _loss functions_, but in this notebook we will use the **root-mean-squared error**, which is defined as

$$\Large
\text{RMSE} = \sqrt{ \frac{1}{n} \sum_{i=1}^n \left ( y_i - \hat{y}_i \right ) ^2 }
$$

where $n$ is the number of observations. The effect of this is to take the mean of the distance of each value of $\hat{y}$ from its corresponding value in $y$; squaring these values keeps them positive, and then we take the square root to correct the units of the error.

**Question 1.3:** Complete the function `rmse` below which computes the root-mean-squared error of the prediction `y_hat` on `y`.

In [ ]:
def rmse(y, y_hat):
    """Computes the RMSE of prediction y_hat based on y"""
    return ...

In [ ]:
grader.check("q1_3")

**Question 1.4:** Use your `rmse` function to compute the RMSE of our prediction `y_hat` based on `y` from above.

In [ ]:
single_var_error = ...
single_var_error

In [ ]:
grader.check("q1_4")

Now that we know how to predict based on and quantify the error of a model, let's write a function that will encapsulate this pipeline for us.

**Question 1.5:** Fill in the function `pred_and_plot` below which models `bill_sep05` based on a column `col`, plots the scatterplot and line of best fit, and computes the RMSE of the model.

_Hint:_ The code from the model above Question 1.3 may be helpful here...

In [ ]:
def pred_and_plot(col):
    """Performs single variable OLS to predict bill_sep05 based on col"""
    x = ...
    y = ...

    alpha = ...
    beta = ...

    y_hat = ...
    
    model_rmse = ...
    
    
    ### DO NOT EDIT THE CODE BELOW ###
    print("RMSE: {:.5f}".format(rmse(y, y_hat)))

    plt.scatter(x, y, color="tab:blue", alpha=0.3)
    plt.plot(x, y_hat, color="tab:red")
    plt.title("Predict September bill with {}".format(col))

**Question 1.6:** Choose a column you think might be related to `bill_sep05` and use your `pred_and_plot` function to determine its prediction RMSE.

In [ ]:
...

In looking through different features, you should have noticed that most of them don't follow a linear relationship very well. In practice, you often need _multiple_ features (explanatory variables) to predict an outcome variable, and it is for this reason that we often use **multiple linear regression** to predict variables.

## Part 2: Guided Multivariate OLS

When we predict a variable $y$ based on some set of $p$ explanatory variables $x$, we create a set of weights $\alpha$ and $\beta_i$ such that we have

$$\Large
y = \alpha + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_p x_p + \varepsilon_i
$$

Because of the error term $\varepsilon_i$, we will instead create predictions $\hat{y}$, such that 

$$\Large
\hat{y} = \alpha + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_p x_p
$$

Let's model the September bill based on the other bills in the data set (April to August). Recall from lecture that we can model an outcome variable `Y` based on columns from our data `defaults` by extracting the values of the table into an array. In the cell below, we create the arrays `X` and `Y`.

In [ ]:
X = defaults.select("bill_aug05", "bill_jul05", "bill_jun05", "bill_may05", "bill_apr05").values
Y = defaults.column("bill_sep05")

Recall that we can fit a multivariate OLS model using `statsmodels` by calling the function `sm.OLS` on the outcome and explanatory variables. In the cell below, we create a model based on _all_ the columns in the table (except, of course, the outcome variable).

In [ ]:
# create an OLS object with the data
model = sm.OLS(Y, sm.add_constant(X))
result = model.fit()
result.summary()

**Question 2.1:** Which variable corresponds to `bill_jul05`?

1. $x_1$
2. $x_2$
3. $x_3$
4. $x_4$
5. $x_5$

Assign your answer to `ans_2_1` below.

In [ ]:
ans_2_1 = ...

In [ ]:
grader.check("q2_1")

**Question 2.2:** What is the standard error of the coefficient of `bill_jun05`?

1. 0.005
2. 0.010
3. 0.039
4. 0.007

Assign your answer to `ans_2_2` below.

In [ ]:
ans_2_2 = ...

In [ ]:
grader.check("q2_2")

Now let's look and see what values our model predicts for our outcome variable. Recall that we can extract the fitted values from the result using `result.fittedvalues`. 

**Question 2.3:** Assign `Y_hat` to the fitted values of `result`. Then assign `multi_rmse` to the RMSE of this prediction based on `Y`.

In [ ]:
Y_hat = ...

multi_rmse = ...
multi_rmse

In [ ]:
grader.check("q2_3")

We see from this RMSE that the prediction is (much) better than the single variable case, but it's still not too good. Let's try and select better features to see if we can lower our RMSE.

**Question 2.4:** What might be a good way to select columns that would lower our prediction RMSE? What kinds of characteristics and relationships should we look for in our feature columns?

_Type your answer here, replacing this text._

Now that you have thought about what might make good features, let's implement a new model to see how well we can do.

**Question 2.5:** Add one more column label to the array `new_features` below. Then fill in the code below to create a new OLS model based on the columns in `new_features`, storing the fitted values in `new_Y_hat`. **Don't forget to apply `sm.add_constant` to `new_X` in your `sm.OLS` call!**

_Hint:_ Our outcome variable `Y` hasn't changed, so we can reuse the same array as earlier.

In [ ]:
new_features = make_array("bill_aug05", "bill_jul05", "paid_aug05", "sex_male", "sex_female", ...)

new_X = defaults.select(...).values

new_model = sm.OLS(...)
new_result = new_model....
new_Y_hat = new_result....
new_Y_hat

In [ ]:
grader.check("q2_5")

Now that we have some predictions, let's look at the accuracy of our model.

**Question 2.6:** Calculate the RMSE of `new_Y_hat` based on `Y` and store this value as `new_rmse`.

In [ ]:
new_rmse = ...
new_rmse

In [ ]:
grader.check("q2_6")

**Question 2.7:** Did the RMSE go up or down in Question 2.6 compared to Question 2.3? Why do you think so?

_Type your answer here, replacing this text._

## Part 3: Unguided Multivariate OLS

In this section of the assignment, you will use `statsmodels` and OLS to create your own model to predict the September 2005 bill. Your model will be scored out of **5 points**, and a portion of your score will be determined based on your RMSE. The scores you will receive are given in the table below.

| RMSE | Score (out of 5) |
|-----|-----|
| $\le$ 20,000 | 6 |
| $\le$ 30,000 | 5 |
| $\le$ 50,000 | 4 |
| $\le \infty$ | 3 |

Note that it is possible to receive a 6 out of 5 for an especially good model, and that as long as you _create a model_, you are guaranteed a 3 out of 5. **To submit your model, you must assign `my_labels` to an array of the columns you want your model to use. You may not use more than 10 columns.** Your model RMSE will be calculated using the following code:

```python
X, Y = defaults.select(my_labels).values, defaults.column("bill_sep05")
model = sm.OLS(Y, sm.add_constant(X))
result = model.fit()
Y_hat = result.fittedvalues
rmse(Y, Y_hat)
```

To select your features, use the widget below to look for correlations between variables and the September 2005 bill. It requires your `pred_and_plot` function to work, so you will need to finish that function before using the widget.

In [ ]:
interact(pred_and_plot, col=Dropdown(options=defaults.labels));

Add and remove cells below as needed, but *make sure you define `my_labels`*. We have provided code for you to create your `X` array; just fill in the `...` in `my_labels` with your columns and use the space at the bottom to work on your model. Good luck!

In [ ]:
my_labels = make_array(...)

my_X = defaults.select(my_labels).values

my_model = sm.OLS(...)
my_result = my_model....
my_Y_hat = my_result....
...

In [ ]:
grader.check("q3")

## Part 4: Reflection

In this section of the assignment, you will answer some conceptual questions about the choices you made in creating your model in Part 3.

**Question 4.1:** Explain one choice you made in selecting features while modeling in Part 3 and why you made it. (Your explanation should take at least a few sentences.)

_Type your answer here, replacing this text._

**Question 4.2:** Use your `pred_and_plot` function in the cell below to generate a visualization that helped you choose a feature in Part 3.

In [ ]:
...

---

### References

* Data from https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#

For your convenience, you can run the cell below to rerun all of your autograder tests.

In [ ]:
grader.check_all()

## Submission

Congrats on finishing another homework notebook! To turn in this homework assignment, **save this file** by going to File > Download As and select **Notebook**; then, run the cell below to generate a PDF of this assignment and download it. Submit this assignment by uploading **BOTH the .ipynb and .pdf files** to Gradescope.

In [ ]:
grader.export("proj03.ipynb")